In [ ]:
import boto3
import sagemaker

session = sagemaker.Session()
bucket = session.default_bucket()

In [ ]:
prefix = 'dogscats'
s3_train_path = 's3://{}/{}/input/train/'.format(bucket, prefix)
s3_val_path   = 's3://{}/{}/input/validation/'.format(bucket, prefix)
s3_output     = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

### Get the name of the image classification algorithm in our region

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

region_name = boto3.Session().region_name
container = get_image_uri(region_name, "image-classification", "latest")
print(container)

### Configure the training job

In [ ]:
role = sagemaker.get_execution_role()

ic = sagemaker.estimator.Estimator(container,
                                   role, 
                                   train_instance_count=1, 
                                   train_instance_type='ml.p2.xlarge',
                                   output_path=s3_output,
                                   sagemaker_session=session)

### Set algorithm parameters

In [ ]:
ic.set_hyperparameters(num_layers=18,               # Train a Resnet-18 model
                       use_pretrained_model=0,      # Train from scratch
                       num_classes=2,               # Dogs and cats
                       num_training_samples=22500,  # Number of training samples
                       mini_batch_size=128,
                       epochs=10)                   # Learn the training samples 10 times

### Set dataset parameters

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_path, 
                                        distribution='FullyReplicated', 
                                        content_type='application/x-recordio',
                                        s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3_val_path,
                                             distribution='FullyReplicated', 
                                             content_type='application/x-recordio', 
                                             s3_data_type='S3Prefix')

s3_channels = {'train': train_data, 'validation': validation_data}

### Train the model

In [ ]:
ic.fit(inputs=s3_channels)

### Deploy the model

In [ ]:
ic_predictor = ic.deploy(initial_instance_count=1,
                         instance_type='ml.t2.medium')

### Download a test image

In [ ]:
# Dog
!wget -O /tmp/test.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/056.dog/056_0010.jpg
file_name = '/tmp/test.jpg'
from IPython.display import Image
Image(file_name)

### Predict test image

In [ ]:
# Load test image from file
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)

# Set content type
ic_predictor.content_type = 'application/x-image'

# Predict image and print result
result = ic_predictor.predict(payload)
print(result)

### Delete endpoint

In [ ]:
ic_predictor.delete_endpoint()